# Project - Segmenting and Clustering Neighborhoods in Toronto, Canada #
## Solving the Problem 1 ##
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.  
To create the above dataframe:  
+ The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
+ Only process the cells that have an assigned borough. Ignore cells with a borough that is **Not assigned**.
+ More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: **Harbourfront** and **Regent Park**. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in **row 11**  in the above table.
+ If a cell has a borough but a **Not assigned**  neighborhood, then the neighborhood will be the same as the borough.
+ Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
+ In the last cell of your notebook, use the **.shape** method to print the number of rows of your dataframe.  
**Note.** I use pandas, the BeautifulSoup package, as a convenient way for me to convert the data in the table on the Wikipedia page to the aforementioned pandas data frame.

In [1]:
import pandas as pd
# !pip install requests
import requests
# !pip install beautifulsoup4
from bs4 import BeautifulSoup as bs

In [2]:
#Scrape the List of postal codes of Canada
page_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(page_url).text

In [3]:
soup = bs(source, 'html.parser')

In [4]:
#Get the first table in the html
data = soup.findAll('table')

In [5]:
def cleaner(seq, value):    
    temp = []                      
    for number in seq:
        if number not in value:
            temp.append(number)
    return temp

In [6]:
# create three lists to store the data into the table
list_postal_code = []
list_borough = []
list_neighborhood = []
# loading data into created lists
data = soup.find('table').find_all('tr')
for row in data:
    cells = row.find_all('td')
    if(len(cells) > 0):
            for cell in cells:
                lst = cleaner(cell.get_text(separator="<br />").strip().split("<br />"),
                              ['','\n','(',')',' / ',', '])
                txt = ''
                for n, value in enumerate(lst):
                    if n == 0 : list_postal_code.append(value)
                    elif n == 1 :list_borough.append(value)
                    else:
                        if len(txt) != 0 : txt += ', ' + value
                        else: txt = value
                if len(txt) == 0 : txt = 'Not assigned' 
                else: txt = txt.replace('(','').replace(')','').replace(' / ',', ').replace('  ',' ').replace(' ,',',').replace(',,','')
                list_neighborhood.append(txt) 

In [8]:
# create a new DataFrame from the three lists
df = pd.DataFrame({"PostalCode": list_postal_code,
                      "Borough": list_borough,
                 "Neighborhood": list_neighborhood})

In [9]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
df = df[df["Borough"] != "Not assigned"]

In [11]:
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Queen's Park,Ontario Provincial Government


In [12]:
df_group = df.groupby("PostalCode")["Neighborhood"].apply(lambda x: "%s" % ', '.join(x))
df_group = df_group.reset_index(drop = False)
df_group.rename(columns = {"Neighborhood":"Neighborhood_joined"},inplace=True)

In [13]:
df_merge = pd.merge(df, df_group, on="PostalCode")
df_merge.drop(["Neighborhood"],axis=1,inplace=True)
df_merge.rename(columns={'Neighbourhood_joined':'Neighbourhood'},inplace=True)

In [14]:
df_merge.head()

,PostalCode,Borough,Neighborhood_joined
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [15]:
df_merge.drop_duplicates(inplace=True)

In [16]:
df_merge.head()

,PostalCode,Borough,Neighborhood_joined
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [17]:
df_merge.shape

(103, 3)

## Solving the Problem 2 ##
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.   
Use the Geocoder package or the csv file to create the dataframe with the fields "PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"

In [18]:
def geocode(postal_code):
    x_y_coords = None
    while (x_y_coords is None):
        g = geocoders.google("{}, Toronto, Ontario".format(postal_code))
        x_y_coords = g.latlng
    latitude = x_y_coords[0]
    longitude = x_y_coords[1]
    return latitude, longitude

In [19]:
geo_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")

In [20]:
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [21]:
geo_df.rename(columns = {"Postal Code":"PostalCode"},inplace = True)
geo_merged = pd.merge(geo_df, df_merge, on = "PostalCode")

In [22]:
geo_merged.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)
geo_merged.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [23]:
geo_data = geo_merged[["PostalCode","Borough","Neighborhood","Latitude","Longitude"]]

In [24]:
geo_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## **Solving the Problem 3** ##
Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.  

Just make sure:  
+ to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
+ to generate maps to visualize your neighborhoods and how they cluster together. 

In [29]:
toronto_data = geo_data[geo_data["Borough"].str.contains("Toronto")]
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth, West Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches, West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [30]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [26]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called _toronto_venues_.

In [31]:
toronto_venues = getNearbyVenues(names = toronto_data['Neighborhood'],
                                 latitudes = toronto_data['Latitude'],
                                 longitudes = toronto_data['Longitude']
                                )

The Beaches
The Danforth, West Riverdale
India Bazaar, The Beaches, West
Studio District
Lawrence Park
Davisville, North
North Toronto, West
Davisville
Moore Park, Summerhill, East
Summerhill, West Rathnelly, South Hill, Forest Hill, SE Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central, Bay Street
Richmond Adelaide King
Harbourfront, East Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill, North & West
The Annex, North Midtown, Yorkville
University of Toronto Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront, West, Bathurst Quay South Niagara, Island airport
Stn A PO Boxes, 25 The Esplanade, Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High P

**Let's check the size of the resulting dataframe**

In [32]:
print(toronto_venues.shape)
toronto_venues.head()

(1577, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


**Let's check how many venues were returned for each neighborhood**

In [33]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,59,59,59,59,59,59
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail, Processing Centre, 969 Eastern, Enclave of M4L",15,15,15,15,15,15
"CN Tower, King and Spadina, Railway Lands, Harbourfront, West, Bathurst Quay South Niagara, Island airport",14,14,14,14,14,14
"Central, Bay Street",61,61,61,61,61,61
Christie,16,16,16,16,16,16
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


**Let's find out how many unique categories can be curated from all the returned venues**

In [34]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


## Analyze Each Neighborhood


In [39]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

**And let's examine the new dataframe size.**

In [40]:
toronto_onehot.shape

(1577, 232)

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.016949,0.0,0.0,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Business reply mail, Processing Centre, 969 Ea...",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.0,0.071429,0.071429,0.071429,0.071429,0.214286,0.142857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
4,"Central, Bay Street",0.016393,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.016393,0.0,0.0,0.016393


**Let's confirm the new size**

In [42]:
toronto_grouped.shape

(38, 232)

**Let's print each neighborhood along with the top 5 most common venues**

In [43]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1        Cocktail Bar  0.07
2              Bakery  0.05
3      Farmers Market  0.03
4  Seafood Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
                   venue  freq
0                   Café  0.13
1         Breakfast Spot  0.09
2  Performing Arts Venue  0.09
3            Coffee Shop  0.09
4              Nightclub  0.04


----Business reply mail, Processing Centre, 969 Eastern, Enclave of M4L----
                  venue  freq
0  Gym / Fitness Center  0.07
1         Garden Center  0.07
2           Pizza Place  0.07
3            Restaurant  0.07
4         Burrito Place  0.07


----CN Tower, King and Spadina, Railway Lands, Harbourfront, West, Bathurst Quay South Niagara, Island airport----
                 venue  freq
0      Airport Service  0.21
1     Airport Terminal  0.14
2  Rental Car Location  0.07
3       Airport Lounge  0.07
4     Sculpture Garden  0.07


----Central, Bay Str

**Let's put that into a pandas dataframe**

First, let's write a function to sort the venues in descending order.

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

**Now let's create the new dataframe and display the top 10 venues for each neighborhood.**

In [46]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Farmers Market,Seafood Restaurant,Beer Bar,Cheese Shop,Pharmacy,Restaurant,Creperie
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Performing Arts Venue,Coffee Shop,Nightclub,Burrito Place,Convenience Store,Stadium,Bar,Bakery
2,"Business reply mail, Processing Centre, 969 Ea...",Gym / Fitness Center,Garden Center,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park,Farmers Market,Fast Food Restaurant,Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Terminal,Rental Car Location,Airport Lounge,Sculpture Garden,Plane,Boat or Ferry,Airport Gate,Airport Food Court,Airport
4,"Central, Bay Street",Coffee Shop,Café,Italian Restaurant,Sandwich Place,Salad Place,Bank,Bubble Tea Shop,Japanese Restaurant,Burger Joint,Thai Restaurant


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [48]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

**Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.**

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Asian Restaurant,Trail,Pub,Health Food Store,Market,Martial Arts School,Lounge,Mediterranean Restaurant,Moroccan Restaurant,Men's Store
41,M4K,East Toronto,"The Danforth, West Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Brewery,Frozen Yogurt Shop,Juice Bar,Spa,Liquor Store
42,M4L,East Toronto,"India Bazaar, The Beaches, West",43.668999,-79.315572,2,Park,Fast Food Restaurant,Sushi Restaurant,Pet Store,Gym,Pub,Movie Theater,Restaurant,Sandwich Place,Brewery
43,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Coffee Shop,Brewery,Bakery,Café,Gastropub,American Restaurant,Yoga Studio,Pet Store,Seafood Restaurant,Cheese Shop
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Molecular Gastronomy Restaurant,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant


In [53]:
from geopy.geocoders import Nominatim

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Finally, let's visualize the resulting clusters

In [54]:
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##### **Thank you !**
This notebook was created by *Dmitry Rabotko.*  
This notebook is part 3 of a course on **Coursera** called _Applied Data Science Capstone_ week 3. 